In [1]:
%load_ext sql
%sql postgresql://admin:admin@db:5432/testdb

Connecting to 'postgresql://admin:***@db:5432/testdb'

In [2]:
%%sql

DROP TABLE IF EXISTS Books;

CREATE TABLE IF NOT EXISTS Books (
    book_id SERIAL primary key,
    title VARCHAR(100),
    author VARCHAR(100),
    genre VARCHAR(50),
    published_year INT,
    price NUMERIC(10, 2),
    stock INT
);

DROP TABLE IF EXISTS Customers;

CREATE TABLE IF NOT EXISTS Customers (
    customer_id SERIAL primary key,
    name VARCHAR(100),
    email VARCHAR(100),
    phone VARCHAR(15),
    city VARCHAR(50),
    country VARCHAR(150)
);

DROP TABLE IF EXISTS Orders;

CREATE TABLE IF NOT EXISTS Orders (
    order_id SERIAL primary key,
    customer_id INT REFERENCES Customers(customer_id),
    book_id INT REFERENCES Books(book_id),
    order_date Date,
    quantity INT,
    total_amount NUMERIC(10, 2)
);

Running query in 'postgresql://admin:***@db:5432/testdb'

RuntimeError: (psycopg2.errors.DependentObjectsStillExist) cannot drop table books because other objects depend on it
DETAIL:  constraint orders_book_id_fkey on table orders depends on table books
HINT:  Use DROP ... CASCADE to drop the dependent objects too.

[SQL: DROP TABLE IF EXISTS Books;]
(Background on this error at: https://sqlalche.me/e/20/2j85)


In [ ]:
%%sql
SELECT * FROM Books;
SELECT * FROM Customers;
SELECT * FROM Orders;

In [ ]:
%%sql
-- use `/copy` if using jupyter notebook, but since we are using docker, we have to past the .csv file inside postgreSQL itself
COPY Books(book_id, title, author, genre, published_year, price, stock)
FROM '/csv/Books.csv'
DELIMITER ','
CSV HEADER;

In [ ]:
%%sql
SELECT * FROM Books;

In [ ]:
%%sql
copy Customers(customer_id, name, email, phone, city, country)
FROM '/csv/Customers.csv'
DELIMITER ','
CSV HEADER;

In [ ]:
%%sql
SELECT * FROM Customers;

In [ ]:
%%sql
copy Orders(order_id, customer_id, book_id, order_date, quantity, total_amount)
FROM '/csv/Orders.csv'
DELIMITER ','
CSV HEADER;

In [ ]:
%%sql
SELECT * FROM orders;

In [ ]:
# -- %%sql
# -- DROP TABLE IF EXISTS Books;
print("You can't delete a table if that is foreign key of some other table")

1. Retrieve all books in the "Fiction" genra

In [ ]:
%%sql
SELECT * FROM Books b WHERE b.genre = 'Fiction'
-- SELECT * FROM Books WHERE genre = 'Fiction'

2. Find books published after the year 1950

In [ ]:
%%sql
SELECT * FROM Books WHERE published_year > 1950;

3. List all customers from the Canada

In [3]:
%%sql
SELECT * FROM Customers
WHERE country = 'Canada'

Running query in 'postgresql://admin:***@db:5432/testdb'

3 rows affected.

customer_id,name,email,phone,city,country
38,Nicholas Harris,christine93@perkins.com,1234567928,Davistown,Canada
415,James Ramirez,robert54@hall.com,1234568305,Maxwelltown,Canada
468,David Hart,stokesrebecca@gmail.com,1234568358,Thompsonfurt,Canada


4. Show orders placed in November 2023

In [4]:
%%sql
SELECT * FROM Orders
WHERE order_date BETWEEN '2023-11-01' AND '2023-11-30';

Running query in 'postgresql://admin:***@db:5432/testdb'

25 rows affected.

order_id,customer_id,book_id,order_date,quantity,total_amount
4,433,343,2023-11-25,7,301.21
19,496,60,2023-11-17,9,316.26
75,291,375,2023-11-30,5,170.75
132,469,333,2023-11-22,7,194.32
137,474,471,2023-11-25,8,363.04
163,207,384,2023-11-23,3,101.76
182,129,293,2023-11-01,7,125.51
200,313,303,2023-11-23,1,6.57
213,325,447,2023-11-17,7,253.75
231,22,384,2023-11-11,1,33.92


5. Retrieve the total stock of books available

In [5]:
%%sql
SELECT SUM(stock) AS total_books_stock FROM Books;

Running query in 'postgresql://admin:***@db:5432/testdb'

1 rows affected.

total_books_stock
25056


6. Find the details of the most expensive book

In [6]:
%%sql
SELECT * FROM Books LIMIT 2;

Running query in 'postgresql://admin:***@db:5432/testdb'

2 rows affected.

book_id,title,author,genre,published_year,price,stock
1,Configurable modular throughput,Joseph Crane,Biography,1949,21.34,100
2,Persevering reciprocal knowledge user,Mario Moore,Fantasy,1971,35.80,19


In [7]:
%%sql
SELECT title as expensive_book FROM Books ORDER BY price DESC LIMIT 1;

Running query in 'postgresql://admin:***@db:5432/testdb'

1 rows affected.

expensive_book
Proactive system-worthy orchestration


7. Show all customers who ordered more than 1 quantity of a book

In [8]:
%%sql
SELECT * FROM Orders LIMIT 1;

Running query in 'postgresql://admin:***@db:5432/testdb'

1 rows affected.

order_id,customer_id,book_id,order_date,quantity,total_amount
1,84,169,2023-05-26,8,188.56


In [12]:
%%sql
SELECT DISTINCT c.name, c.customer_id
FROM Orders
INNER JOIN Customers c ON Orders.quantity > 1; 

Running query in 'postgresql://admin:***@db:5432/testdb'

500 rows affected.

name,customer_id
Mark Wilson,124
Peter Krueger,73
Rebecca Buckley,152
Manuel Lewis,63
Sarah Cameron,290
Kurt Greer,366
Elijah Baker,230
Shawn Gonzales,381
Carol Petersen,276
William Cook,205


8. Retrieve all orders where the total amount exceeds $20.

In [18]:
%%sql
SELECT * FROM orders
WHERE total_amount > 20
ORDER BY total_amount DESC
LIMIT 10;

Running query in 'postgresql://admin:***@db:5432/testdb'

10 rows affected.

order_id,customer_id,book_id,order_date,quantity,total_amount
368,17,488,2023-05-17,10,491.50
91,226,142,2024-10-13,10,489.60
247,77,142,2024-03-07,10,489.60
211,397,209,2024-01-17,10,486.70
492,284,120,2024-04-28,10,480.30
481,405,353,2023-07-07,10,469.70
135,174,99,2024-01-24,10,469.30
178,70,298,2023-05-03,10,466.60
491,425,72,2023-10-14,10,465.40
459,163,422,2024-08-17,10,459.10


9. List all genres available in the Books table:

In [21]:
%%sql
SELECT DISTINCT genre FROM books;

Running query in 'postgresql://admin:***@db:5432/testdb'

7 rows affected.

genre
Romance
Biography
Mystery
Fantasy
Fiction
Non-Fiction
Science Fiction


10. Find the book with the lowest stock:

In [26]:
%%sql
SELECT * FROM Books
WHERE stock < 10
ORDER BY stock
LIMIT 1;

Running query in 'postgresql://admin:***@db:5432/testdb'

1 rows affected.

book_id,title,author,genre,published_year,price,stock
44,Networked systemic implementation,Ryan Frank,Science Fiction,1965,13.55,0


11. Calculate the total revenue generated from all orders:

In [29]:
%%sql
SELECT SUM(total_amount) as Total_revenue FROM Orders;

Running query in 'postgresql://admin:***@db:5432/testdb'

1 rows affected.

total_revenue
75628.66


12. Retrieve the total number of books sold for each genre:

In [35]:
%%sql
SELECT DISTINCT genre 
FROM Books;

Running query in 'postgresql://admin:***@db:5432/testdb'

7 rows affected.

genre
Romance
Biography
Mystery
Fantasy
Fiction
Non-Fiction
Science Fiction


In [37]:
%%sql
SELECT * FROM Orders Limit 3;

Running query in 'postgresql://admin:***@db:5432/testdb'

3 rows affected.

order_id,customer_id,book_id,order_date,quantity,total_amount
1,84,169,2023-05-26,8,188.56
2,137,301,2023-01-23,10,216.60
3,216,261,2024-05-27,6,85.50


In [40]:
%%sql
SELECT b.genre, o.quantity
FROM Books b
JOIN Orders o ON o.book_id = b.book_id;

Running query in 'postgresql://admin:***@db:5432/testdb'

500 rows affected.

genre,quantity
Biography,8
Fantasy,10
Science Fiction,6
Mystery,7
Romance,7
Romance,5
Romance,6
Biography,4
Romance,9
Mystery,4


In [44]:
%%sql
SELECT b.genre, SUM(o.quantity) as Total_books_sold
FROM Orders o
JOIN Books b ON o.book_id = b.book_id
GROUP BY b.genre;

Running query in 'postgresql://admin:***@db:5432/testdb'

7 rows affected.

genre,total_books_sold
Romance,439
Biography,285
Mystery,504
Fantasy,446
Fiction,225
Non-Fiction,351
Science Fiction,447


13. Find the average price of books in the "Fantasy" genre:

In [45]:
%%sql
SELECT * FROM Books;

Running query in 'postgresql://admin:***@db:5432/testdb'

500 rows affected.

book_id,title,author,genre,published_year,price,stock
1,Configurable modular throughput,Joseph Crane,Biography,1949,21.34,100
2,Persevering reciprocal knowledge user,Mario Moore,Fantasy,1971,35.80,19
3,Streamlined coherent initiative,Derrick Howard,Non-Fiction,1913,15.75,27
4,Customizable 24hour product,Christopher Andrews,Fiction,2020,43.52,8
5,Adaptive 5thgeneration encoding,Juan Miller,Fantasy,1956,10.95,16
6,Advanced encompassing implementation,Bryan Morgan,Biography,1985,6.56,2
7,Open-architected exuding structure,Jacqueline Young,Romance,1927,43.63,95
8,Persistent local encoding,Troy Cox,Science Fiction,2019,48.99,84
9,Optimized interactive challenge,Colin Buckley,Fantasy,1987,14.33,70
10,Ergonomic national hub,Samantha Ruiz,Mystery,2015,24.63,25


In [54]:
%%sql
SELECT b.genre, AVG(b.price) as Average_Price
FROM Books b
WHERE b.genre = 'Fantasy'
GROUP BY b.genre;

Running query in 'postgresql://admin:***@db:5432/testdb'

1 rows affected.

genre,average_price
Fantasy,25.9816901408450704


14. List customers who have placed at least 2 orders:

In [71]:
%%sql
SELECT o.customer_id, c.name, COUNT(o.order_id) AS Order_count
FROM Orders o
JOIN Customers c ON o.customer_id = c.customer_id
GROUP BY o.customer_id, c.name
HAVING COUNT(order_id) >= 2;

Running query in 'postgresql://admin:***@db:5432/testdb'

139 rows affected.

customer_id,name,order_count
225,Christopher Mccullough,2
418,Kiara Blankenship MD,3
322,William Cameron,3
325,Emily Vargas,4
376,Justin Donaldson,2
486,Melanie Kelly,2
461,Crystal Pierce,3
2,Crystal Clements,2
149,Jason Robinson,3
173,Victoria Dixon,2


15. Find the most frequently ordered book:

In [76]:
%%sql
SELECT o.book_id, b.title, COUNT(o.order_id) AS Order_count
FROM Orders o
JOIN books b ON o.book_id = b.book_id
GROUP BY o.book_id, b.title
ORDER BY order_count DESC LIMIT 1;

Running query in 'postgresql://admin:***@db:5432/testdb'

1 rows affected.

book_id,title,order_count
88,Robust tangible hardware,4


16. Show the top 3 most expensive books of 'Fantasy' Genre:

In [79]:
%%sql
SELECT * FROM Books
WHERE genre = 'Fantasy'
ORDER BY price DESC LIMIT 3;

Running query in 'postgresql://admin:***@db:5432/testdb'

3 rows affected.

book_id,title,author,genre,published_year,price,stock
240,Stand-alone content-based hub,Lisa Ellis,Fantasy,1957,49.90,41
462,Innovative 3rdgeneration database,Allison Contreras,Fantasy,1988,49.23,62
238,Optimized even-keeled analyzer,Sherri Griffith,Fantasy,1975,48.97,72


17. Retrieve the total quantity of books sold by each author:

In [80]:
%%sql
SELECT b.author, SUM(o.quantity) AS Total_Books_Sold
FROM Orders o
JOIN Books b ON o.book_id = b.book_id
GROUP BY b.author;

Running query in 'postgresql://admin:***@db:5432/testdb'

314 rows affected.

author,total_books_sold
Jared Cortez,10
Tracy Parker,11
Taylor Wang,9
Cathy Knight,6
Bianca Matthews,3
Douglas Malone,6
James Alvarado,9
Betty Cross,6
Michael Hill,20
Steven Mcdonald,15


18. List the cities where customers who spent over $30 are located:

In [87]:
%%sql
SELECT DISTINCT c.city, total_amount -- o.total_amount
FROM Orders o
JOIN Customers c ON o.customer_id = c.customer_id
WHERE o.total_amount > 300;

Running query in 'postgresql://admin:***@db:5432/testdb'

65 rows affected.

city,total_amount
Freemanland,469.70
East David,301.21
East Richardburgh,330.64
South Rachelview,440.70
South Timothy,394.70
Port Jessica,346.50
Matthewfurt,328.50
Lindaburgh,325.92
Natashaville,399.04
Gonzalestown,326.62


19. Find the customer who spent the most on orders:

In [92]:
%%sql
SELECT c.customer_id, c.name, SUM(o.total_amount) AS Total_Spent
FROM Orders o
JOIN Customers c ON o.customer_id = c.customer_id
GROUP BY c.customer_id, c.name
ORDER BY Total_Spent DESC
LIMIT 1;

Running query in 'postgresql://admin:***@db:5432/testdb'

1 rows affected.

customer_id,name,total_spent
457,Kim Turner,1398.90


20. Calculate the stock remaining after fulfilling all orders:

In [106]:
%%sql
SELECT b.book_id, b.title, b.stock, COALESCE(SUM(o.quantity), 0) AS Order_quantity, b.stock - COALESCE(SUM(o.quantity), 0) AS Remaining_Quantity
FROM Books b
LEFT JOIN Orders o ON b.book_id = o.book_id
GROUP BY b.book_id;

Running query in 'postgresql://admin:***@db:5432/testdb'

500 rows affected.

book_id,title,stock,order_quantity,remaining_quantity
87,Decentralized actuating analyzer,98,0,98
184,Enterprise-wide solution-oriented challenge,22,9,13
477,Distributed cohesive approach,17,13,4
273,Devolved zero administration process improvement,23,24,-1
394,Integrated well-modulated Graphic Interface,59,1,58
51,Visionary optimizing project,95,3,92
272,Intuitive 4thgeneration intranet,79,10,69
70,Customer-focused context-sensitive process improvement,19,7,12
190,Cross-platform even-keeled focus group,4,0,4
350,Exclusive 24hour groupware,49,14,35
